In [1]:
import time
import random
import os
import subprocess


class ExecutionTime:
    def __init__(self):
        self.start_time = time.time()

    def duration(self):
        return time.time() - self.start_time
    
    def reset(self):
        self.start_time = time.time()

# ---- parameters ----#

In [2]:
# select the input file to be runned
input_file = "rna.fasta"

# ---- run old Lara code ---- #

In [3]:
lara_exe = os.path.join(os.getcwd(), "lara-1.3.2", "lara")
lara_path = os.path.join(os.getcwd(), "lara-1.3.2")
fasta_path = os.path.join(os.getcwd(), "inputs", input_file)

cmd_lara = lara_exe + " -i " + fasta_path

#enter the lara folder
os.chdir(lara_path)

#run the timer
timer = ExecutionTime()
#execute the old lara
proc = subprocess.Popen([cmd_lara], stdout=subprocess.PIPE, shell=True)
#output the old lara results
(out, err) = proc.communicate()
#print the time required for the operation
print('Old Lara computation finished in {} seconds.'.format(timer.duration()))
#print (out)

# Extract the fasta output only from the Old Lara output
j = 0
out_fasta = list()
out_splitted = out.split('\n')
for i in range(len(out_splitted)):
    if(len(out_splitted[i])>0):
        if(out_splitted[i][0]=='>'):
            out_fasta.append(out_splitted[i])
            out_fasta.append(out_splitted[i+1])
            i = i+1
            
#leave the old lara folder
os.chdir("..")

Old Lara computation finished in 237.487302065 seconds.


## Convert the Fasta file format in the ClustalW (to be complient with the formata available in the refs of bralibase) 


In [22]:
import io
from Bio import AlignIO

def iterable_to_stream(iterable, buffer_size=io.DEFAULT_BUFFER_SIZE):
    """
    Lets you use an iterable (e.g. a generator) that yields bytestrings as a read-only
    input stream.

    The stream implements Python 3's newer I/O API (available in Python 2's io module).
    For efficiency, the stream is buffered.
    """
    class IterStream(io.RawIOBase):
        def __init__(self):
            self.leftover = None
        def readable(self):
            return True
        def readinto(self, b):
            try:
                l = len(b)  # We're supposed to return at most this much
                chunk = self.leftover or next(iterable)
                output, self.leftover = chunk[:l], chunk[l:]
                b[:len(output)] = output
                return len(output)
            except StopIteration:
                return 0    # indicate EOF
    return io.BufferedReader(IterStream(), buffer_size=buffer_size)

input_handle = iterable_to_stream((str(string + '\n').encode('utf8') for string in out_fasta))
output_handle = open("./tmp/tcoffeeLaraOld.clustalw", "w")

alignments = AlignIO.parse(input_handle, "fasta")
AlignIO.write(alignments, output_handle, "clustal")
#AlignIO.write(alignments, output_handle2, "stockholm")
output_handle.close()

# ---- run Lara2 ---- #

In [23]:
lara2_path = "../cmake-build-release/laragu"
fasta_path2 = os.path.join("inputs", input_file)

cmd_lara2 = lara2_path + " -i " + fasta_path2 + " -t 2 -a -lgo -2 -lge -1 -lbm 1 -ssc 10 -tcm 2 -tb 0.2 -v 0 -td ./tmp"
#TODO check the lara program with this input file
#run the timer
timer = ExecutionTime()
#execute the lara2
proc = subprocess.Popen([cmd_lara2], stdout=subprocess.PIPE, shell=True)
#output the lara2 results
(out, err) = proc.communicate()
#print the time required for the operation
print('Lara2 computation finished in {} seconds.'.format(timer.duration()))
#print (out)
print (cmd_lara2)

Lara2 computation finished in 2.94378399849 seconds.
../cmake-build-release/laragu -i inputs/rna.fasta -t 2 -a -lgo -2 -lge -1 -lbm 1 -ssc 10 -tcm 2 -tb 0.2 -v 0 -td ./tmp


# ---- run old t-coffee src ---- #
tcoffee_location -in=/tmp/tcoffeLara.lib -case=upper -output fasta -clean_seq_name 1 -outfile /tmp/tcoffeeLara.aln -newtree /tmp/tcoffeeLara.dnd 2> /dev/null > /dev/null 

In [25]:
tcoffee_path = "./lara-1.3.2/t_coffee/t_coffee_5.05"
in_file_path =  "./tmp/tcoffeLara.lib"

cmd_part1 = tcoffee_path + " -in=" + in_file_path + " -case=upper -output clustal -clean_seq_name 1 -outfile "

out_file_path = "./tmp/tcoffeeOldLara.aln"
tree_file_path = "./tmp/tcoffeeOldLara.dnd"

#os.mkdir("mkdir ~/.t_coffee 2> /dev/null")
cmd_tcoffee = cmd_part1 + out_file_path  + " -newtree " + tree_file_path #+ " 2> /dev/null > /dev/null "

#run the timer
timer = ExecutionTime()
#execute the tcoffee
proc = subprocess.Popen([cmd_tcoffee], stdout=subprocess.PIPE, shell=True)
#output the tcoffee results
(out, err) = proc.communicate()
#print the time required for the operation
print('TCoffee computation finished in {} seconds.'.format(timer.duration()))
#print (out)
print (cmd_tcoffee)

TCoffee computation finished in 29.3483650684 seconds.
./lara-1.3.2/t_coffee/t_coffee_5.05 -in=./tmp/tcoffeLara.lib -case=upper -output clustal -clean_seq_name 1 -outfile ./tmp/tcoffeeOldLara.aln -newtree ./tmp/tcoffeeOldLara.dnd


# ---- run new seqan::t-coffee src ---- #

In [26]:
tcoffee_path = "seqan_tcoffee"
in_file_path = "./tmp/tcoffeLara.lib"

cmd_part1new = tcoffee_path + " -s ./inputs/" + input_file + " -l " + in_file_path + " -m global -a rna "

out_file_path_tc2 = "./tmp/tcoffeeLara.msf"

cmd_tcoffeeNew = cmd_part1new + " -o " + out_file_path_tc2 + " -b wavg "

#run the timer
timer = ExecutionTime()
#execute the tcoffee
proc = subprocess.Popen([cmd_tcoffeeNew], stdout=subprocess.PIPE, shell=True)
#output the tcoffee results
(out, err) = proc.communicate()
#print the time required for the operation
print('TCoffee computation finished in {} seconds.'.format(timer.duration()))
print (out)
print (cmd_tcoffeeNew)

TCoffee computation finished in 0.234289884567 seconds.

seqan_tcoffee -s ./inputs/rna.fasta -l ./tmp/tcoffeLara.lib -m global -a rna  -o ./tmp/tcoffeeLara.msf -b wavg 


## Compute consensus structure with: i) RNAalifold, ii) ipknot, iii) centroid-rna

## RNAalifold consensus structure

## Ipknot consensus structure

# Visualize the computed structures vit Jviz and ...

# Test AlignIO module for fileformat conversion Fasta -> Clustalw

In [20]:
from Bio import AlignIO
align = AlignIO.read("./tmp/old/tcoffeeLara.aln", "fasta")
print(align)

SingleLetterAlphabet() alignment with 11 rows and 532 columns
uauggg---aucccuagc--ggggguaa----------------...--a AE0066621/4933-4623
gaggaa---aguccagc---------------------------...--a AF1923621/1-260
gaggaa---acugcagc---------------------------...--a AF1923631/1-259
--------------------------------------------...--- AF1923491/1-262
g-------------------------------------------...--- AF1923501/1-26
uagggg---agccua-aca-ggggguua----------------...a-u L135971/422-736
ggcagagagagcccaguuc-ccgugcccga--------------...cuu M610031/281-714
ggcagagagagcccggcca-ccgcggcguaccgacaggcacaca...ugu U429811/1-475
ggcagagagagcccggcca-ccgcggcgugccagca-gcacgca...--- U429821/1-358
ggcagagggagcccggcugcccgug-----ccguga--------...gua U429831/1-375
uagggg---agccua-aca-gggggcca----------------...--- AF1217731/1-303


In [23]:
input_handle = open("./tmp/old/tcoffeeLara.aln", "rU")
output_handle = open("./tmp/old/tcoffeeLara.clustalw", "w")
#output_handle2 = open("./tmp/old/tcoffeeLara.stk", "w")

alignments = AlignIO.parse(input_handle, "fasta")
AlignIO.write(alignments, output_handle, "clustal")
#AlignIO.write(alignments, output_handle2, "stockholm")

output_handle.close()
input_handle.close()
